# 利用SqueezeNet结构训练cumt数据
## 2017年11月8日

In [1]:
# %load_ext autoreload
# %autoreload 2
# %reload_ext autoreload

import tensorflow as tf
import numpy as np
from cumt_eye import Cumt_itraker
import matplotlib.pyplot as plt
import cv2
from Cumt_SqueezeNet import squeezenet
init=tf.global_variables_initializer()

sess=tf.InteractiveSession()
cumt_data=np.load('cumt_data.pkl')
index=np.arange(cumt_data['data'].shape[0])
np.random.shuffle(index)
data=cumt_data['data'][index]
label=cumt_data['labels'][index]
del cumt_data

#cumt rgb 相片均值
cumt_picmean=[103.939, 116.779, 123.68]

#### 将图片从 （32,128,3）转换成（128,128,3），有可能会导致精度下降，非必要


In [2]:
resize_img=[]
for i in range(data.shape[0]):
    img=data[i]+cumt_picmean
    img=cv2.resize(img.astype('uint8'),(128,128))
    resize_img.append(img.astype('uint8'))
    if i%10000==0:
        print (i)
data=None
resize_img=np.array(resize_img).astype('uint8')

0
10000
20000
30000
40000
50000
60000
70000


#### 常规设置占位符，训练节点等

In [4]:
with tf.name_scope('PlaceHolder'):
    X=tf.placeholder(shape=[None,128,128,3],dtype=tf.float32)
    Y=tf.placeholder(shape=[None,10],dtype=tf.float32)
    LR=tf.placeholder(dtype=tf.float32)
with tf.name_scope('model'):
    model=squeezenet(X,sess)
    y_score=model.score
with tf.name_scope('LOSS'):
    LOSS=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_score,labels=Y))
    tf.summary.scalar('loss',LOSS)
with tf.name_scope('TRAIN'):
    TRAIN=tf.train.AdamOptimizer(LR).minimize(LOSS)
with tf.name_scope('ACCURACY'):
    acc_count=tf.equal(tf.arg_max(y_score,1),tf.arg_max(Y,1))
    ACCURACY=tf.reduce_mean(tf.cast(acc_count,tf.float32))
    tf.summary.scalar('acc',ACCURACY)
writer_tr=tf.summary.FileWriter('./mylog/squeezenet/train')
writer_te=tf.summary.FileWriter('./mylog/squeezenet/test',sess.graph)
writer_val=tf.summary.FileWriter('./mylog/squeezenet/val')
merge=tf.summary.merge_all()

building graph


#### train,test,val数据生成

In [6]:
data=resize_img
index=np.arange(data.shape[0])
np.random.shuffle(index)
tr_index=index[:int(data.shape[0]*0.9)]
te_index=index[int(data.shape[0]*0.9):]
tr_data,tr_label=data[tr_index],label[tr_index]
te_data,te_label=data[te_index],label[te_index]
del data
del label
del resize_img
print(tr_data.shape,te_data.shape)

addr=r'D:\Proj_DL\Code\Proj_EyeTraker\Proj_iTraker\CUMT_iTraker\test_video\101603\101603_z.pkl'
val_d=np.load(addr)
val_data=[]
for v in range(val_d['images'].shape[0]):
    img_=cv2.resize(val_d['images'][v],(128,128))
    val_data.append(img_)
val_data=np.array(val_data)
val_label=val_d['labels']
del val_d
print(val_data.shape)

(67346, 128, 128, 3) (7483, 128, 128, 3)


#### 开始训练

In [10]:
init=tf.global_variables_initializer()
sess.run(init)

In [ ]:
lr_=1e-3
batchsize=64
for i in range(1000*6*12):
    mask=np.random.choice(tr_data.shape[0],batchsize,replace=False)
    x_,y_=tr_data[mask]-cumt_picmean,tr_label[mask]
    feed_dict={X:x_,Y:y_,LR:lr_}
    sess.run(TRAIN,feed_dict=feed_dict)
    if i%10==0:
        feed_dict={X:x_,Y:y_}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_tr.add_summary(m_,i)
        print('epoch:{},train loss:{},train accuracy:{}'.format(i,loss_,acc_))
    if i%30==0:
        mask=np.random.choice(te_data.shape[0],batchsize,replace=False)
        x_,y_=te_data[mask]-cumt_picmean,te_label[mask]
        feed_dict={X:x_,Y:y_}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_te.add_summary(m_,i)
        print('--epoch:{},test loss:{},test accuracy:{}'.format(i,loss_,acc_))
    if i%100==0:
        mask=np.random.choice(val_data.shape[0],batchsize,replace=False)
        x_,y_=val_data[mask]-cumt_picmean,val_label[mask]
        feed_dict={X:x_,Y:y_}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_val.add_summary(m_,i)
        print('@@epoch:{},val loss:{},val accuracy:{}'.format(i,loss_,acc_))

epoch:0,train loss:2.284879684448242,train accuracy:0.140625
--epoch:0,test loss:2.313260078430176,test accuracy:0.0625
@@epoch:0,val loss:2.2761244773864746,val accuracy:0.171875
epoch:10,train loss:2.2994980812072754,train accuracy:0.125
epoch:20,train loss:2.29630970954895,train accuracy:0.15625
epoch:30,train loss:2.3021130561828613,train accuracy:0.109375
--epoch:30,test loss:2.299588203430176,test accuracy:0.140625
epoch:40,train loss:2.301260232925415,train accuracy:0.109375
epoch:50,train loss:2.302359104156494,train accuracy:0.140625
epoch:60,train loss:2.3041205406188965,train accuracy:0.078125
--epoch:60,test loss:2.302917957305908,test accuracy:0.09375
epoch:70,train loss:2.302932024002075,train accuracy:0.109375
epoch:80,train loss:2.302196741104126,train accuracy:0.109375
epoch:90,train loss:2.3037943840026855,train accuracy:0.078125
--epoch:90,test loss:2.304551362991333,test accuracy:0.0625
epoch:100,train loss:2.302884101867676,train accuracy:0.09375
@@epoch:100,val lo

epoch:910,train loss:2.304311513900757,train accuracy:0.078125
epoch:920,train loss:2.2993617057800293,train accuracy:0.15625
epoch:930,train loss:2.302194356918335,train accuracy:0.109375
--epoch:930,test loss:2.301413059234619,test accuracy:0.125
epoch:940,train loss:2.3038551807403564,train accuracy:0.078125
epoch:950,train loss:2.301501750946045,train accuracy:0.125
epoch:960,train loss:2.3029732704162598,train accuracy:0.09375
--epoch:960,test loss:2.302957534790039,test accuracy:0.09375
epoch:970,train loss:2.3017489910125732,train accuracy:0.125
epoch:980,train loss:2.3036298751831055,train accuracy:0.0625
epoch:990,train loss:2.3024816513061523,train accuracy:0.109375
--epoch:990,test loss:2.303072452545166,test accuracy:0.0625
epoch:1000,train loss:2.3035404682159424,train accuracy:0.03125
@@epoch:1000,val loss:2.3005611896514893,val accuracy:0.25
epoch:1010,train loss:2.301934003829956,train accuracy:0.15625
epoch:1020,train loss:2.303035259246826,train accuracy:0.078125
--ep

epoch:1820,train loss:2.298856258392334,train accuracy:0.140625
epoch:1830,train loss:2.302130699157715,train accuracy:0.109375
--epoch:1830,test loss:2.2968521118164062,test accuracy:0.15625
epoch:1840,train loss:2.308539867401123,train accuracy:0.046875
epoch:1850,train loss:2.3035075664520264,train accuracy:0.09375
epoch:1860,train loss:2.3032703399658203,train accuracy:0.09375
--epoch:1860,test loss:2.3010454177856445,test accuracy:0.125
epoch:1870,train loss:2.3040924072265625,train accuracy:0.078125
epoch:1880,train loss:2.297161102294922,train accuracy:0.203125
epoch:1890,train loss:2.3012611865997314,train accuracy:0.125
--epoch:1890,test loss:2.3020858764648438,test accuracy:0.109375
epoch:1900,train loss:2.303194761276245,train accuracy:0.09375
@@epoch:1900,val loss:2.2969040870666504,val accuracy:0.203125
epoch:1910,train loss:2.3040688037872314,train accuracy:0.078125
epoch:1920,train loss:2.303945779800415,train accuracy:0.078125
--epoch:1920,test loss:2.3030967712402344,t

epoch:2720,train loss:2.304764747619629,train accuracy:0.15625
epoch:2730,train loss:2.302657127380371,train accuracy:0.125
--epoch:2730,test loss:2.301715135574341,test accuracy:0.109375
epoch:2740,train loss:2.300588846206665,train accuracy:0.109375
epoch:2750,train loss:2.2978975772857666,train accuracy:0.09375
epoch:2760,train loss:2.303342342376709,train accuracy:0.171875
--epoch:2760,test loss:2.303971290588379,test accuracy:0.078125
epoch:2770,train loss:2.301316738128662,train accuracy:0.140625
epoch:2780,train loss:2.301251173019409,train accuracy:0.09375
epoch:2790,train loss:2.3021841049194336,train accuracy:0.078125
--epoch:2790,test loss:2.304563045501709,test accuracy:0.140625
epoch:2800,train loss:2.30098819732666,train accuracy:0.15625
@@epoch:2800,val loss:2.2980849742889404,val accuracy:0.15625
epoch:2810,train loss:2.2998135089874268,train accuracy:0.140625
epoch:2820,train loss:2.302067756652832,train accuracy:0.09375
--epoch:2820,test loss:2.305480718612671,test ac

epoch:3620,train loss:2.3068289756774902,train accuracy:0.109375
epoch:3630,train loss:2.3102731704711914,train accuracy:0.0625
--epoch:3630,test loss:2.3031930923461914,test accuracy:0.109375
epoch:3640,train loss:2.3038289546966553,train accuracy:0.078125
epoch:3650,train loss:2.308277130126953,train accuracy:0.0625
epoch:3660,train loss:2.298304557800293,train accuracy:0.140625
--epoch:3660,test loss:2.299645185470581,test accuracy:0.109375
epoch:3670,train loss:2.3101534843444824,train accuracy:0.03125
epoch:3680,train loss:2.294032096862793,train accuracy:0.140625
epoch:3690,train loss:2.30373477935791,train accuracy:0.078125
--epoch:3690,test loss:2.2997806072235107,test accuracy:0.078125
epoch:3700,train loss:2.2974982261657715,train accuracy:0.09375
@@epoch:3700,val loss:2.2919397354125977,val accuracy:0.25
epoch:3710,train loss:2.3076257705688477,train accuracy:0.078125
epoch:3720,train loss:2.299354314804077,train accuracy:0.09375
--epoch:3720,test loss:2.304774761199951,test

epoch:4520,train loss:2.3059239387512207,train accuracy:0.09375
epoch:4530,train loss:2.3051443099975586,train accuracy:0.09375
--epoch:4530,test loss:2.29986572265625,test accuracy:0.109375
epoch:4540,train loss:2.303229331970215,train accuracy:0.09375
epoch:4550,train loss:2.30387544631958,train accuracy:0.0625
epoch:4560,train loss:2.2967820167541504,train accuracy:0.125
--epoch:4560,test loss:2.303297281265259,test accuracy:0.078125
epoch:4570,train loss:2.301499366760254,train accuracy:0.140625
epoch:4580,train loss:2.303576707839966,train accuracy:0.125
epoch:4590,train loss:2.2964565753936768,train accuracy:0.125
--epoch:4590,test loss:2.296483039855957,test accuracy:0.140625
epoch:4600,train loss:2.302952527999878,train accuracy:0.078125
@@epoch:4600,val loss:2.293022632598877,val accuracy:0.25
epoch:4610,train loss:2.299351692199707,train accuracy:0.140625
epoch:4620,train loss:2.2982306480407715,train accuracy:0.125
--epoch:4620,test loss:2.302172899246216,test accuracy:0.140

In [ ]:
#保存模型
saver=tf.train.Saver()
saver.save(sess,'./model_save/SqueezeNet/model_984.ckpt',global_step=i)

In [ ]:
addr=r'D:\Proj_DL\Code\Proj_EyeTraker\Proj_iTraker\CUMT_iTraker\test_video\101603\101603_z.pkl'
val_d=np.load(addr)

In [ ]:
val_d['images'].shape

In [ ]:
img_val=cv2.resize(val_d['images'][0],(128,128))
plt.imshow(img_val.astype('uint8'))
plt.show()

In [ ]:
map_lab={ 
        '7':4,'8':3,'9':2,
        '4':7,'5':6,'6':5,
        '1':10,'2':9,'3':8,
'0':1}
         

In [ ]:
prob=tf.nn.softmax(y_score)

In [ ]:
val_data=[]
for v in range(val_d['images'].shape[0]):
    img_=cv2.resize(val_d['images'][v],(128,128))
    val_data.append(img_.)

In [ ]:
val_data=np.array(val_data)

In [ ]:
val_data.shape

In [ ]:
x_.shape

In [ ]:
val_d['labels'][mask].shape

In [ ]:
x_=val_data-cumt_picmean
mean_sum=0
for k in range(50):
    mask=np.random.choice(range(val_data.shape[0]),128,replace=False)
    acc_=sess.run(ACCURACY,feed_dict={X:x_[mask],Y:val_d['labels'][mask]})
    #print(acc_)
    mean_sum+=acc_
print('mean accuracy:')
print(mean_sum/50)

In [ ]:
acc_

In [ ]:
import os
from scipy.misc import imread
from scipy.misc import imresize

In [ ]:
def getimg(addr):
    VGG_MEAN = [103.939, 116.779, 123.68]
    a=imread(addr)
    plt.imshow(a)
    plt.show()
    b=imresize(a-VGG_MEAN,(32,128,3)).reshape((1,32,128,3))
    return b

In [ ]:
img_dir=r'D:\Proj_DL\Code\Proj_EyeTraker\eye_val\441_5.jpg'
b=getimg(img_dir)

In [ ]:
p_=sess.run(prob,feed_dict={X:b,DROPOUT:1.,BN_FLAG:False,LR:1e-3})

In [ ]:
list(p_)

In [ ]:
np.argmax(p_)+1

In [ ]:
#计算所有可训练参数的数量
sum_=0
for s in list(tf.trainable_variables()):
    s=str(s)
    shape_=s[s.index('(')+1:s.index(')')].split(',')
    s_=1
    #print(shape_)
    for i in shape_:
        if i =='':continue
        #print(i)
        s_*=int(i)
    sum_+=s_

In [ ]:
sum_